<a href="https://colab.research.google.com/github/SH0123/DeepLearning/blob/master/logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [0]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [8]:
x_data = torch.FloatTensor([[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]])
y_data = torch.FloatTensor([[0], [0], [0], [1], [1], [1]])

model = BinaryClassifier()

optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = model(x_data)

    cost = F.binary_cross_entropy(hypothesis, y_data)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_data
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100
        ))

Epoch    0/100 Cost: 0.778947 Accuracy 33.33%
Epoch   10/100 Cost: 0.606802 Accuracy 66.67%
Epoch   20/100 Cost: 0.446548 Accuracy 66.67%
Epoch   30/100 Cost: 0.376169 Accuracy 83.33%
Epoch   40/100 Cost: 0.318945 Accuracy 83.33%
Epoch   50/100 Cost: 0.268428 Accuracy 83.33%
Epoch   60/100 Cost: 0.222594 Accuracy 100.00%
Epoch   70/100 Cost: 0.183695 Accuracy 100.00%
Epoch   80/100 Cost: 0.158160 Accuracy 100.00%
Epoch   90/100 Cost: 0.144616 Accuracy 100.00%
Epoch  100/100 Cost: 0.134716 Accuracy 100.00%
